# 第7章：数据规整化：清理、转换、合并、重塑

# 主要内容
# 2.数据变换
2.1 删除重复值  
2.2 用函数和映射来转换数据  
2.3 替换值  
2.4 重命名Axis Indexes  
2.5 离散化和分箱处理  
2.6 检测和过滤异常值  
2.7 排序和随机抽样  
2.8 计算指示器/虚拟变量

# 2.1 删除重复值
1. duplicated()判断是否为重复值
2. drop_duplicates()删除重复值

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


duplicated()函数返回一个boolean Series，表示一个row是否是重复的（根据前一行来判断）：

In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

两种方法删除重复的行：  
1. drop_duplicaters()函数  
2. 先用duplicated()判断，再过滤

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [8]:
data[data.duplicated() == False]

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


也可以检测指定列的重复值

In [9]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [10]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicated默认保留第一次观测到的数值组合。设置keep='last'能返回最后一个：

In [11]:
data.drop_duplicates('k1', keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


# 2.2 用函数和映射来转换数据
使用map()函数

也就是根据变量来计算新变量。  
例子:有不同种类的肉，假如想加入一列，表示每一种肉的来源是什么动物，可以写一个映射。

In [12]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


新建一个字典

In [13]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

先用str.lower把data中'food'变量的所有值变为小写

In [14]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [15]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


上面的过程也可以用匿名函数表示

In [28]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

# 2.3 替换值
replace()函数

fillna是一个特殊换的替换操作。map可以用于修改一个object里的部分值，但是replace能提供一个更简单和更灵活的方法做到这点

In [30]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

用replace，会产生一个新series(也可以使用inplace=True来替换)

In [31]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [32]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [33]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [34]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

注意：data.replace方法和data.str.replace方法是不同的，后者会对string进行element-wise替换

# 2.4 重命名Axis Indexes
1. map()函数
2. rename()函数

像是series里的value一样，axis label也能类似地是函数或映射来转换

In [35]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [36]:
data.index.map(lambda x: x[:4].upper())

array(['OHIO', 'COLO', 'NEW '], dtype=object)

In [38]:
str_to_int = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4}

也可以改变列名

In [54]:
data.columns.map(lambda x: str_to_int[x])

array([1, 2, 3, 4], dtype=int64)

也可以用rename函数来改变

In [56]:
data.rename(index={'Ohio': 'INDIANA'},
            columns={'three': 'pekaboo'})

,one,two,pekaboo,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [57]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [58]:
data.rename(index={'Ohio': 'INDIANA'},
            columns={'three': 'pekaboo'}, inplace = True)
data

,one,two,pekaboo,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


# 2.5 离散化和分箱处理
1. cut()函数
2. qcut()

连续型数据经常被离散化或分散成bins（分箱）来分析。  
例如，将年龄划分为年龄组

In [59]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

我们把这些分到四个bin里，19~25, 26~35, 36~60, >60。可以用pandas里的cut

In [60]:
bins = [18, 25, 35, 60, 100]
cuts = pd.cut(ages, bins)
cuts

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [63]:
type(cuts)

pandas.core.categorical.Categorical

返回的是一个特殊的Categorical object。

In [62]:
cuts.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [65]:
cuts.categories

Index(['(18, 25]', '(25, 35]', '(35, 60]', '(60, 100]'], dtype='object')

可以利用cut函数的right参数来设置区间开或者闭

In [66]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

也可以用一个list或数组给labels选项来设定bin的名字

In [67]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [68]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

均为分为4个bins

In [70]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)
# precision表示小数点后精确的位数

[(0.51, 0.75], (0.26, 0.51], (0.75, 0.99], (0.51, 0.75], (0.75, 0.99], ..., (0.26, 0.51], (0.023, 0.26], (0.51, 0.75], (0.51, 0.75], (0.26, 0.51]]
Length: 20
Categories (4, object): [(0.023, 0.26] < (0.26, 0.51] < (0.51, 0.75] < (0.75, 0.99]]

qcut，会按照数据的分位数来分箱。取决于数据的分布，用cut通常不能保证每一个bin有一个相同数量的数据点。而qcut是按百分比来切的，所以可以得到等数量的bins

In [71]:
data = np.random.randn(1000)

In [72]:
cuts = pd.qcut(data, 4) # 用分位数来分箱
cuts

[(-0.669, -0.00287], [-3.271, -0.669], [-3.271, -0.669], (-0.669, -0.00287], (-0.00287, 0.723], ..., (0.723, 3.223], [-3.271, -0.669], [-3.271, -0.669], (0.723, 3.223], (0.723, 3.223]]
Length: 1000
Categories (4, object): [[-3.271, -0.669] < (-0.669, -0.00287] < (-0.00287, 0.723] < (0.723, 3.223]]

In [73]:
pd.value_counts(cuts)

(0.723, 3.223]        250
(-0.00287, 0.723]     250
(-0.669, -0.00287]    250
[-3.271, -0.669]      250
dtype: int64

qcut保证每个bin的数量相同

在cut中我们可以自己指定百分比

In [74]:
cuts2 = pd.cut(data, [0, 0.1, 0.5, 0.9, 1.]) # 累进的百分比
cuts2

[NaN, NaN, NaN, NaN, (0.1, 0.5], ..., (0.5, 0.9], NaN, NaN, (0.5, 0.9], NaN]
Length: 1000
Categories (4, object): [(0, 0.1] < (0.1, 0.5] < (0.5, 0.9] < (0.9, 1]]

In [76]:
pd.value_counts(cuts2)

(0.1, 0.5]    151
(0.5, 0.9]    122
(0, 0.1]       31
(0.9, 1]       28
dtype: int64

# 2.6 检测和过滤异常值

假设我们想要找一个列中，绝对值大于3的数字

In [77]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.018195,-0.010863,-0.006510,0.000177
std,1.007058,0.965237,1.010872,1.029482
min,-3.020160,-2.622653,-4.053474,-3.245082
25%,-0.692574,-0.643543,-0.683962,-0.664180
50%,0.025268,-0.022122,-0.038335,0.004046
75%,0.725869,0.657606,0.655301,0.749876
max,3.113817,3.397569,2.917369,3.059622


In [78]:
data.head()

,0,1,2,3
0,0.410078,-0.140566,-0.381601,-1.813685
1,-2.879825,-0.919372,1.036801,-1.782116
2,0.707190,-0.201026,1.059751,2.551028
3,-0.364375,0.743011,-1.098001,0.009760
4,-1.461871,-2.235008,-0.751024,0.467488


In [79]:
col = data[2]
col.head()

0   -0.381601
1    1.036801
2    1.059751
3   -1.098001
4   -0.751024
Name: 2, dtype: float64

In [80]:
col[np.abs(col) > 3]

90    -3.221640
346   -3.103465
434   -3.140828
604   -3.390137
717   -4.053474
Name: 2, dtype: float64

选中所有值大于3的行

In [83]:
data[(np.abs(data) > 0.5)].head()

,0,1,2,3
0,NaN,NaN,NaN,-1.813685
1,-2.879825,-0.919372,1.036801,-1.782116
2,0.707190,NaN,1.059751,2.551028
3,NaN,0.743011,-1.098001,NaN
4,-1.461871,-2.235008,-0.751024,NaN


In [84]:
data[21:23]

,0,1,2,3
21,0.161102,1.560605,2.011078,-0.380360
22,-0.994342,-1.150045,-0.670179,-0.813992


In [85]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.018195,-0.010863,-0.006510,0.000177
std,1.007058,0.965237,1.010872,1.029482
min,-3.020160,-2.622653,-4.053474,-3.245082
25%,-0.692574,-0.643543,-0.683962,-0.664180
50%,0.025268,-0.022122,-0.038335,0.004046
75%,0.725869,0.657606,0.655301,0.749876
max,3.113817,3.397569,2.917369,3.059622


np.sign(data)会根据值的正负号来得到1或-1

In [86]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,1.0,-1.0
2,1.0,-1.0,1.0,1.0
3,-1.0,1.0,-1.0,1.0
4,-1.0,-1.0,-1.0,1.0


# 2.7 排列和随机采样
1. numpy.random.permutation()函数可进行排序
2. permutation产生的数组可用于take函数
2. sample()函数可进行抽样

In [87]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [88]:
sampler = np.random.permutation(5)
sampler

array([4, 3, 2, 1, 0])

这个数组能被用在基于iloc上的indexing或take函数：

In [89]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3


In [90]:
df.take([4,2,3,1,0])

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3


In [91]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
1,4,5,6,7


In [92]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

1    7
0    5
0    5
3    6
1    7
2   -1
1    7
4    4
0    5
1    7
dtype: int64

# 2.8 计算指示器/虚拟变量
get_dummies()函数

虚拟变量是指取值为0和1的值，如将“季度”用虚拟变量表示，则应该设置三个虚拟变量（当回归带截距项时，设立n-1个虚拟变量，其中n为类别的数量，若设n个虚拟变量，则会陷入“虚拟变量陷阱”）：X、Y、Z，X=1(第一季度),X=0(其他)。Y=1(第二季度),Y=0(其他)，Z=1(第三季度),Z=0(其他)

为什么要设置虚拟变量呢？不能直接用“季节”这个变量吗？  
首先，“季度”的取值为字符，不能用于回归分析。  
其次，假如“季度”取值为1，2，3，4分别代表四季，能直接用于分析吗？  
答案是不能。因为这样会把类别变量默认为连续变量，也就是说，软件自动认为“第二季度”比“第一季度”多1，“第三季度”比“第二季度”多1.....。这样显然是不合理的。  
因此，需要设置虚拟变量。  

In [93]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


可以用2.2中的方法来设置虚拟变量，但是用get_dummies()函数可以直接得到虚拟变量

In [94]:
pd.get_dummies(df['key'])

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


如果我们想要给column加一个前缀， 可以用data.get_dummies里的prefix参数来实现

In [95]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [98]:
type(dummies)

pandas.core.frame.DataFrame

In [100]:
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0
